## Installations

In [1]:
!pip3 install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 105.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 84.0 MB/s eta 0:00:00


## Read data from S3

In [1]:
import boto3
import string
import random

In [6]:
# Create an SSM client
ssm = boto3.client('ssm', region_name="us-east-1")

# Fetch the parameter value
response = ssm.get_parameter(
    Name='/kaggle/bucket_name',
    WithDecryption=False
)

bucket_name = response['Parameter']['Value']

In [7]:
s3 = boto3.client('s3')

In [9]:
def read_file_from_s3(bucket_name, file_name):
    obj = s3.get_object(Bucket = bucket_name, Key = file_name)
    data = obj['Body'].read().decode('utf-8')
    return data

In [11]:
file_name = "arrange/eng_news_2023_1M/eng_news_2023_1M/eng_news_2023_1M-sentences.txt"
file_content = read_file_from_s3(bucket_name, file_name)

In [15]:
cleaned = [line.split("\t")[1].lower().strip() for line in file_content.splitlines()]
input_texts = [x.translate(str.maketrans('', '', string.punctuation)) for x in cleaned]

In [16]:
input_texts[16985]

'after a 20year battle with debilitating arthritis dunedin man bill napier was willing to try anything'

## Preprocess data

In [9]:
from transformers import BertTokenizer, TFBertModel

2024-12-05 10:39:02.556374: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-05 10:39:13.535594: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-05 10:39:13.636179: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-05 10:39:28.770364: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-05 10:39:44.024994: W tensorflow/compiler/tf

In [10]:
input_texts = input_texts[0:5000]

NameError: name 'input_texts' is not defined

In [ ]:
target_texts_token = [x.split(" ") for x in input_texts]
target_texts = [" ".join(random.sample(x, len(x))) for x in target_texts_token]

In [ ]:
# Charger le tokenizer et le modèle BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
input_encodings = tokenizer(input_texts, padding=True, truncation=True, return_tensors="tf")
output_encodings = tokenizer(target_texts, padding=True, truncation=True, return_tensors="tf")

In [ ]:
input_encodings["input_ids"].shape

In [ ]:
output_encodings["input_ids"].shape

## Define the model

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam

In [ ]:
class BertSeq2Seq(tf.keras.Model):
    def __init__(self, bert_model, vocab_size, hidden_size):
        super(BertSeq2Seq, self).__init__()
        self.bert = bert_model
        self.decoder = tf.keras.layers.LSTM(hidden_size, return_sequences=True, return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size, activation="softmax")

    def call(self, inputs):
        input_ids, attention_mask = inputs["input_ids"], inputs["attention_mask"]
        # Encode with BERT
        encoder_output = self.bert(input_ids, attention_mask=attention_mask)[0]

        # Decode
        decoder_output, _, _ = self.decoder(encoder_output)
        output = self.dense(decoder_output)

        return output

In [11]:
# Load pre-trained BERT model
bert_model = TFBertModel.from_pretrained("bert-base-uncased")

# Instantiate the seq2seq model
vocab_size = tokenizer.vocab_size
hidden_size = bert_model.config.hidden_size
model = BertSeq2Seq(bert_model, vocab_size, hidden_size)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

2024-12-05 10:40:23.018295: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertMo

NameError: name 'tokenizer' is not defined

## Train model

In [12]:
# Load pre-trained BERT model
bert_model = TFBertModel.from_pretrained("bert-base-uncased")

# Instantiate the seq2seq model
vocab_size = tokenizer.vocab_size
hidden_size = bert_model.config.hidden_size
model = BertSeq2Seq(bert_model, vocab_size, hidden_size)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

NameError: name 'tokenizer' is not defined

In [13]:
inputs = {
    "input_ids": input_encodings["input_ids"],
    "attention_mask": input_encodings["attention_mask"]
}
labels = output_encodings["input_ids"]

# Train the model
model.fit(inputs,
          labels,
          batch_size=64,
          epochs=3,
          validation_split=0.2)


NameError: name 'input_encodings' is not defined

 ## Predictions

In [ ]:
def predict_sentence(model, tokenizer, input_sentence, max_length=20):
    inputs = tokenizer(input_sentence, return_tensors="tf", padding="max_length", truncation=True, max_length=max_length)
    prediction = model.predict({"input_ids": inputs["input_ids"], "attention_mask": inputs["attention_mask"]})
    predicted_ids = tf.argmax(prediction, axis=-1).numpy()
    return tokenizer.decode(predicted_ids[0], skip_special_tokens=True)


In [ ]:
jumbled_sentence = "sleeps cat under tree the"
corrected_sentence = predict_sentence(model, tokenizer, jumbled_sentence)
print("Reordered Sentence:", corrected_sentence)
